In [ ]:
import ee
import geemap
import os
import numpy as np
import pandas as pd

# ========== Authenticate and initialize Earth Engine ==========
ee.Authenticate()
ee.Initialize()

In [ ]:
# ========== Set user-defined parameters ==========
start_date = '2018-05-03'
end_date = '2018-10-04'
output_folder = './chaohu_landsat8'
os.makedirs(output_folder, exist_ok=True)

# Draw or load your ROI (region of interest)
Map = geemap.Map()
roi = Map.user_roi  # Manually draw or load via the GUI

In [ ]:
# ========== Function: Apply cloud and snow mask ==========
def mask_cloud_snow(image):
    """
    Apply bitmask to remove clouds and snow using QA_PIXEL band.
    """
    qa = image.select('QA_PIXEL')
    cloud_bit = 1 << 3
    snow_bit = 1 << 5
    mask = qa.bitwiseAnd(cloud_bit).eq(0).And(
           qa.bitwiseAnd(snow_bit).eq(0))
    return image.updateMask(mask)

# ========== Function: Scale reflectance bands ==========
def scale_sr_bands(image):
    """
    Scale Landsat SR bands to reflectance values using official scale factors.
    """
    sr = image.select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])
    scaled = sr.multiply(0.0000275).add(-0.2)
    return image.addBands(scaled, overwrite=True)

# ========== Function: Compute Floating Algae Index (FAI) ==========
def compute_fai(image):
    """
    Calculate Floating Algae Index (FAI) from red, NIR, and SWIR bands.
    """
    fai = image.expression(
        'nir - red - (swir - red) * (865 - 655) / (1610 - 655)', {
            'red': image.select('SR_B4'),
            'nir': image.select('SR_B5'),
            'swir': image.select('SR_B6')
        }
    )
    return image.addBands(fai.rename('FAI'))

# ========== Function: Load and preprocess Landsat 8 SR image collection ==========
def get_landsat8_collection(start, end, roi):
    """
    Return a preprocessed Landsat 8 image collection (masked, scaled, with FAI).
    """
    collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterDate(start, end) \
        .filterBounds(roi) \
        .map(mask_cloud_snow) \
        .map(scale_sr_bands) \
        .map(compute_fai)
    return collection

In [ ]:
# ========== Load dataset ==========
dataset = get_landsat8_collection(start_date, end_date, roi)

# ========== Retrieve acquisition dates ==========
date_list = dataset.aggregate_array('DATE_ACQUIRED').getInfo()
print(f"✅ Total available dates: {len(date_list)}")

# ========== Define failed index list to redownload ==========
fail_list = [2, 5, 9]  # Replace with your actual failed indices

# ========== Loop through failed dates and re-download ==========
for i in fail_list:
    date = date_list[i]
    filename = os.path.join(output_folder, f"{date}_landsat8_chaohu.tif")
    try:
        image = dataset.filter(ee.Filter.eq('DATE_ACQUIRED', date)).first()
        geemap.download_ee_image(
            image=image,
            filename=filename,
            crs='EPSG:4326',
            region=roi,
            scale=30
        )
        print(f"✅ Saved: {filename}")
    except Exception as e:
        print(f"❌ Error on {date}: {e}")